In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score

# Apriori Algorithm


In [2]:
# Apriori algorithm
# We combine the above steps to generate frequent itemsets with k+1 items
# from frequent itemsets with k items
# We continue this process until we get no frequent itemsets with k+1 items
# We then combine the frequent itemsets with k items to generate association rules
# We continue this process until we get no association rules
# We then combine the association rules to generate association rules with k+1 items


# Function to generate frequent itemsets with 1 item (initialisation)
def generate_freq_1_itemsets(data, min_support, combined_freq_itemsets):

    # Count the number of 0s and 1s in each column
    # The number of 1s is the number of times each item appears
    value_counts = data.apply(pd.value_counts)

    # Get the frequent itemsets with count greater than or equal to min_support
    columns = data.columns
    frequent_itemsets = {}
    for column in columns:
        # Append the itemset and its count to the dictionary if the count is greater than or equal to min_support
        if value_counts[column][1] >= min_support:
            frequent_itemsets[column] = value_counts[column][1]
            # frequent_itemsets.append((column, value_counts[column][1]))
            # data.drop(column, axis=1, inplace=True)

    dummy_dict = frequent_itemsets.copy()
    for key, item in dummy_dict.copy().items():
        # For dummy data
        # dummy_dict[(tuple(key))] = dummy_dict.pop(key)
        # For real data
        dummy_dict[(key,)] = dummy_dict.pop(key)
    # print(dummy_dict)

    combined_freq_itemsets.update(dummy_dict)

    # print(frequent_itemsets)
    return frequent_itemsets


# Function to generate frequent itemsets with k+1 items
def generate_k_plus_1_candidate_itemsets(frequent_itemsets, k):
    # Generate all possible combinations of frequent itemsets with k+1 items

    # If k = 1, we do not need to merge the combinations
    if k == 1:
        combinations = []
        combinations.append(list(itertools.combinations(frequent_itemsets.keys(), k+1)))
        return combinations
    
    else:
        # Merge the combinations if the first k-1 items are the same
        # and the last item is different
        # This is done to generate combinations with k+1 items
        # from combinations with k items
        # Compare first k-1 items of each combination
        # If they are the same, merge them
        # If they are not the same, do not merge them
        # The merged combinations are stored in a dictionary
        merged_combinations = {}
        

        for index, combination1 in enumerate(frequent_itemsets.keys()):
            for combination2 in list(frequent_itemsets.keys())[index+1:]:
                # Check if the first k-1 items are the same
                if combination1[:-1] == combination2[:-1]:
                    # Check if the last item is different
                    if combination1[-1] != combination2[-1]:
                        # Merge the combinations
                        merged_combinations[combination1 + (combination2[-1],)] = 0

    
        return merged_combinations

# Function to count the number of occurences of each combination in the candidate itemsets
def k_plus_1_itemsets_support_counting(k_plus_1_candidate_itemsets, k, data):
    # If k = 1, we need to convert the list of lists of tuples to a list of tuples
    if k == 1:
        k_plus_1_candidate_itemsets = k_plus_1_candidate_itemsets[0]

    # Count the number of occurences of each combination in the data
    candidate_itemsets_count = {}
    for candidate_itemset in k_plus_1_candidate_itemsets:
        # Using groupby and size to count the number of occurences of each combination
        # Resetting the index to get the count of each combination as a column in the dataframe
        test = data.groupby(list(candidate_itemset)).size().reset_index(name='count')

        # Append the combination and its count to the dictionary
        # The count of each combination is the last value in the count column
        # Moreover, we need to check whether the last row is a combination of 1s instead of 1s and 0s
        # If it is a combination of 1s, then we append the combination and its count to the dictionary
        # Otherwise, we do not append it to the dictionary
        # if test[test.columns[0]].iloc[-1] == 1 and test[test.columns[1]].iloc[-1] == 1:
        #     candidate_itemsets_count[candidate_itemset] = test['count'].iloc[-1]
        num_ones = 0
        for i in range(len(test.columns)-1):
            if test[test.columns[i]].iloc[-1] != 1:
                break
            else:
                num_ones += 1
                continue
            
        if num_ones == len(test.columns)-1:
            candidate_itemsets_count[candidate_itemset] = test['count'].iloc[-1]

    return candidate_itemsets_count


def candidate_elimination(combinations_count, min_support, combined_freq_itemsets):
    
    # Prune the combinations with count less than min_support
    for combination in combinations_count.copy().keys():
        if combinations_count[combination] < min_support:
            combinations_count.pop(combination)
    
    combined_freq_itemsets.update(combinations_count)
    return combinations_count

def generate_rules(combined_freq_itemsets, min_confidence, target):
    # Generate rules for frequent itemsets with k+1 items with min confidence
    # The rules are generated by splitting the combination into two parts
    rules = {}
    for key in combined_freq_itemsets.keys():
        
        for i in range(1, len(key)+1):  # range will return the values 1,2,3,4 in this loop
            combinations = []
            combinations.append(list(itertools.combinations(key, i)))
            # print(target, key, combinations)
            if combinations:
                combinations = combinations[0]
                for combination in combinations:
                    
                    # Convert the combination to a tuple if it is a string
                    if type(combination) == str:
                        combination = (combination,)
                    
                    # Check if the target is in the combination
                    if target != None:
                        # Continue to the next combination if the target is not in the combination
                        if target not in combination or len(combination) == 1:
                            continue
                        
                        # Split the combination into two parts
                        # The first part is the antecedent and the second part is the consequent
                        # The antecedent is the combination without the target
                        # The consequent is the target

                        temp_target = (target,)
                        
                        # In order to keep the correct order of the items in the combination
                        difference = set(combination) - set(temp_target)
                        antecedent = tuple(item for item in combination if item in difference)
                        consequent = temp_target
                        
                        confidence = combined_freq_itemsets[combination] / combined_freq_itemsets[antecedent]

                    # If the target is None, then we do not need to assign the target to the consequent
                    else:
                        antecedent = combination
                        difference = set(key) - set(combination)
                        consequent = tuple(item for item in key if item in difference)
                        if consequent == ():
                            continue
                    # print(target, combination)
                    # print("Combinations is ", combinations, "Combination is: ", combination, "Antecedent is: ", antecedent, "Consequent is: ", consequent)
                        confidence = combined_freq_itemsets[key] / combined_freq_itemsets[antecedent]
                        
                    # print('key: ', key, 'antecedent: ', antecedent, 'consequent: ', consequent, 'confidence: ', confidence)
                    if confidence >= min_confidence:
                        rules[(antecedent, consequent)] = confidence
                        
        # Split the combination into two parts
        # The first part is the antecedent and the second part is the consequent
        # for i in range(1, len(key)):
        #     antecedent_1 = key[:i]
        #     consequent_1 = key[i:]

        #     antecedent_2 = key[i:]
        #     consequent_2 = key[:i]
        #     # Calculate the confidence of the rule
        #     # Confidence = support of combination / support of antecedent
        #     confidence_1 = combined_freq_itemsets[key] / combined_freq_itemsets[antecedent_1]
        #     confidence_2 = combined_freq_itemsets[key] / combined_freq_itemsets[antecedent_2]

        #     print(antecedent_1, consequent_1, confidence_1)
        #     print(antecedent_2, consequent_2, confidence_2)
        #     # Check if the confidence is greater than min_confidence
        #     if confidence_1 >= min_confidence:
        #         # Append the rule to the rules dictionary
        #         rules[(antecedent_1, consequent_1)] = confidence_1

        #     if confidence_2 >= min_confidence:
        #         # Append the rule to the rules dictionary
        #         rules[(antecedent_2, consequent_2)] = confidence_2
                
    return rules
    

def my_apriori(data, min_support, min_confidence, target=None):
    
    # Combined dictionary of frequent itemsets
    combined_freq_itemsets = {}

    # Get frequent 1 itemsets
    frequent_1_itemsets = generate_freq_1_itemsets(data, min_support, combined_freq_itemsets)

    k_plus_1_candidate_itemsets = None
    k_plus_1_itemsets_support_count = None
    k_plus_1_frequent_itemsets = None
    
    k = 1

    while True:
        # print(k)
        if k == 1:
            k_plus_1_candidate_itemsets = generate_k_plus_1_candidate_itemsets(frequent_1_itemsets, k)
        else:
            k_plus_1_candidate_itemsets = generate_k_plus_1_candidate_itemsets(k_plus_1_frequent_itemsets, k)
        # print(combined_freq_itemsets)
        # print(k_plus_1_candidate_itemsets)
        k_plus_1_itemsets_support_count = k_plus_1_itemsets_support_counting(k_plus_1_candidate_itemsets, k, data)
        
        k_plus_1_frequent_itemsets = candidate_elimination(k_plus_1_itemsets_support_count, min_support, combined_freq_itemsets)
        # print(k_plus_1_frequent_itemsets)
        k += 1
        # print('k: ', k)
        # If there are no frequent itemsets with k+1 items, break
        if len(k_plus_1_frequent_itemsets) == 0:
            break

    # Generate rules for frequent itemsets with k+1 items with min confidence
    # The rules are generated by splitting the combination into two parts
    rules = generate_rules(combined_freq_itemsets, min_confidence, target)
    
    return combined_freq_itemsets, rules


# Agglomerative Clustering

In [4]:
# Normalize or standardize your data if necessary
def agglomerative(data):
    scaler = StandardScaler()
    X = scaler.fit_transform(data)
    # Define the number of clusters (k) you want to create
    n_clusters = 2 
    # Initialize and fit the KMeans model
    agg = AgglomerativeClustering(n_clusters=n_clusters, linkage="single")
    agg.fit(X)

    # Assign cluster labels to your data
    cluster_labels = agg.labels_

    # Compute the silhouette score for your clustering
    silhouette_avg = silhouette_score(X, cluster_labels)
    ari = adjusted_rand_score(true_labels, cluster_labels)
    nmi = normalized_mutual_info_score(true_labels, cluster_labels)

    print("Silhouette Score:", silhouette_avg)
    print("ARI:", ari)
    print("NMI:", nmi)

# Titanic Data

In [3]:
df = pd.read_csv('dataset/titanic/train.csv')

true_labels = df['Survived']

titanic_df = df
titanic_df.drop(columns=['PassengerId', 'Name','SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'], inplace=True)
titanic_df['Age'] = pd.cut(titanic_df['Age'], bins=[0, 21, 55, 80], labels=['Child', 'Adult', 'Elderly'])
titanic_df = pd.get_dummies(titanic_df)
titanic_df = pd.get_dummies(titanic_df, columns=['Pclass'])

In [5]:
# original dataset
agglomerative(titanic_df)

891
Silhouette Score: 0.3475127070800924
ARI: -0.006620125534683856
NMI: 0.0016988137512052015


In [6]:
#Run apriori algo on titanic dataset
combined_freq_itemsets, rules = my_apriori(titanic_df, 100, 0.4, target='Survived')

In [7]:
# print the frequent itemset
count = 0
for key, value in combined_freq_itemsets.items():
    if 'Survived' in key:
        print(key, value)
        count += 1
print(count)


('Survived',) 342
('Survived', 'Sex_female') 233
('Survived', 'Sex_male') 109
('Survived', 'Age_Adult') 191
('Survived', 'Embarked_S') 217
('Survived', 'Pclass_1') 136
('Survived', 'Pclass_3') 119
('Survived', 'Sex_female', 'Age_Adult') 132
('Survived', 'Sex_female', 'Embarked_S') 140
('Survived', 'Age_Adult', 'Embarked_S') 136
10


In [8]:
# original dataset + frequent itemset
frequent_titanic_df = titanic_df.copy()
for key in combined_freq_itemsets:
    # Create a new column with 1 if all specified columns in the combination are 1, and 0 otherwise
    if 'Survived' in key:
        combo_name = '_'.join(key)  # Create a column name based on the combination
        frequent_titanic_df[combo_name] = frequent_titanic_df[list(key)].all(axis=1).astype(int)
frequent_titanic_df

,Survived,Sex_female,Sex_male,Age_Child,Age_Adult,Age_Elderly,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,...,Pclass_3,Survived_Sex_female,Survived_Sex_male,Survived_Age_Adult,Survived_Embarked_S,Survived_Pclass_1,Survived_Pclass_3,Survived_Sex_female_Age_Adult,Survived_Sex_female_Embarked_S,Survived_Age_Adult_Embarked_S
0,0,0,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,1,1,0,0,1,0,1,0,0,1,...,0,1,0,1,0,1,0,1,0,0
2,1,1,0,0,1,0,0,0,1,0,...,1,1,0,1,1,0,1,1,1,1
3,1,1,0,0,1,0,0,0,1,1,...,0,1,0,1,1,1,0,1,1,1
4,0,0,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
887,1,1,0,1,0,0,0,0,1,1,...,0,1,0,0,1,1,0,0,1,0
888,0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
889,1,0,1,0,1,0,1,0,0,1,...,0,0,1,1,0,1,0,0,0,0


In [9]:
# original dataset + frequent itemset
agglomerative(frequent_titanic_df)

891
Silhouette Score: 0.34306862096394125
ARI: 0.16825401853457436
NMI: 0.22194103633642098


In [10]:
# only frequent itemset
frequent_df = pd.DataFrame()
for key in combined_freq_itemsets:
    # Create a new column with 1 if all specified columns in the combination are 1, and 0 otherwise
    if 'Survived' in key:
        combo_name = '_'.join(key)  # Create a column name based on the combination
        frequent_df[combo_name] = frequent_titanic_df[list(key)].all(axis=1).astype(int)
frequent_df

,Survived,Survived_Sex_female,Survived_Sex_male,Survived_Age_Adult,Survived_Embarked_S,Survived_Pclass_1,Survived_Pclass_3,Survived_Sex_female_Age_Adult,Survived_Sex_female_Embarked_S,Survived_Age_Adult_Embarked_S
0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,0,1,0,1,0,0
2,1,1,0,1,1,0,1,1,1,1
3,1,1,0,1,1,1,0,1,1,1
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
886,0,0,0,0,0,0,0,0,0,0
887,1,1,0,0,1,1,0,0,1,0
888,0,0,0,0,0,0,0,0,0,0
889,1,0,1,1,0,1,0,0,0,0


In [11]:
# only frequent itemset
agglomerative(frequent_df)

891
Silhouette Score: 0.5957735808938553
ARI: 0.16825401853457436
NMI: 0.22194103633642098


# Business Review Data

In [36]:
df = pd.read_csv('dataset/business/sliced_review_business_3.csv')
df = df[df.columns[1:]]
df

,good review,useful,funny,cool,is_open,business_stars_Bad,business_stars_Average,business_stars_Good,review_count_Low,review_count_Medium,review_count_High
0,1,0,0,0,1,0,0,1,1,0,0
1,0,0,0,0,1,0,0,1,1,0,0
2,0,1,1,0,1,0,0,1,1,0,0
3,0,0,0,0,1,0,1,0,1,0,0
4,0,0,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
69898,1,0,0,0,1,0,0,1,1,0,0
69899,1,0,0,0,1,0,0,1,1,0,0
69900,0,1,0,0,1,1,0,0,1,0,0
69901,0,0,0,0,0,0,1,0,1,0,0


In [38]:
true_labels = df['good review']

In [39]:
# original dataset
agglomerative(df)

69903
Silhouette Score: 0.5122964859824961
ARI: 0.00017556780483242614
NMI: 2.0807549200003233e-05


In [30]:
#Run apriori algo on healthcare dataset
combined_freq_itemsets, rules = my_apriori(df, 10000, 0.4, target='good review')

In [31]:
# print the frequent itemset
count = 0
for key, value in combined_freq_itemsets.items():
    if 'good review' in key:
        print(key, value)
        count += 1
print(count)


('good review',) 32373.0
('good review', 'is_open') 27709
('good review', 'business_stars_Good') 29513
('good review', 'review_count_Low') 27306
('good review', 'is_open', 'business_stars_Good') 25325
('good review', 'is_open', 'review_count_Low') 22928
('good review', 'business_stars_Good', 'review_count_Low') 24527
('good review', 'is_open', 'business_stars_Good', 'review_count_Low') 20616
8


In [41]:
# original dataset + frequent itemset
frequent_business_df = df.copy()
for key in combined_freq_itemsets:
    # Create a new column with 1 if all specified columns in the combination are 1, and 0 otherwise
    if 'good review' in key:
        combo_name = '_'.join(key)  # Create a column name based on the combination
        frequent_business_df[combo_name] = frequent_business_df[list(key)].all(axis=1).astype(int)
frequent_business_df

,good review,useful,funny,cool,is_open,business_stars_Bad,business_stars_Average,business_stars_Good,review_count_Low,review_count_Medium,review_count_High,good review_is_open,good review_business_stars_Good,good review_review_count_Low,good review_is_open_business_stars_Good,good review_is_open_review_count_Low,good review_business_stars_Good_review_count_Low,good review_is_open_business_stars_Good_review_count_Low
0,1,0,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1
1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69898,1,0,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1
69899,1,0,0,0,1,0,0,1,1,0,0,1,1,1,1,1,1,1
69900,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
69901,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [42]:
# original dataset + frequent itemset
agglomerative(frequent_business_df)

69903
Silhouette Score: 0.3943348236143989
ARI: 0.00017556780483242614
NMI: 2.0807549200003233e-05


In [43]:
# only frequent itemset
frequent_df = pd.DataFrame()
for key in combined_freq_itemsets:
    # Create a new column with 1 if all specified columns in the combination are 1, and 0 otherwise
    if 'good review' in key:
        combo_name = '_'.join(key)  # Create a column name based on the combination
        frequent_df[combo_name] = frequent_business_df[list(key)].all(axis=1).astype(int)
frequent_df

,good review,good review_is_open,good review_business_stars_Good,good review_review_count_Low,good review_is_open_business_stars_Good,good review_is_open_review_count_Low,good review_business_stars_Good_review_count_Low,good review_is_open_business_stars_Good_review_count_Low
0,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
69898,1,1,1,1,1,1,1,1
69899,1,1,1,1,1,1,1,1
69900,0,0,0,0,0,0,0,0
69901,0,0,0,0,0,0,0,0


In [44]:
# only frequent itemset
agglomerative(frequent_df)

69903
Silhouette Score: 0.7668579946553834
ARI: 0.4398714847415524
NMI: 0.46735033274029736
